In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/Naive_CD20_pos_B/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/Naive_CD20_pos_B/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/2.seurat_pipeline.240506/RNA_count_matrix/Naive_CD20_pos_B.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.Naive_CD20_pos_B.240517/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Prepare background data

In [6]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 151478 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100028794-100029147      *
       [2]     chr1 100029423-100029644      *
       [3]     chr1 100037570-100039259      *
       [4]     chr1 100046452-100047257      *
       [5]     chr1 100063663-100063892      *
       ...      ...                 ...    ...
  [151474]     chrX     9986627-9987251      *
  [151475]     chrX     9988877-9989092      *
  [151476]     chrX       999373-999875      *
  [151477]     chrX     9995615-9996371      *
  [151478]     chrX     9997597-9997902      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [7]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100028794-100029147,30,20,31,20,27,16,2,23,20,11,⋯,25,24,21,22,42,40.39548,354,chr1,17,0.18080513
chr1-100029423-100029644,6,12,12,8,16,15,6,21,9,14,⋯,10,7,17,15,39,47.29730,222,chr1,14,0.07315914
chr1-100037570-100039259,101,75,100,56,82,160,134,118,90,156,⋯,91,60,103,103,96,58.87574,1690,chr1,2042,0.97295317
chr1-100046452-100047257,41,17,46,68,39,28,5,64,36,29,⋯,65,56,61,79,137,37.22084,806,chr1,332,0.85529912
chr1-100063663-100063892,15,10,25,5,12,15,5,16,25,15,⋯,15,4,8,24,9,56.08696,230,chr1,17,0.18080513
chr1-100064772-100065272,18,22,35,30,39,38,3,47,27,32,⋯,30,21,36,50,49,47.90419,501,chr1,29,0.41657534
chr1-100065371-100065580,15,6,3,9,9,37,1,30,4,4,⋯,1,5,31,5,47,42.38095,210,chr1,14,0.07315914
chr1-100132485-100133460,48,50,61,29,44,92,58,75,67,70,⋯,53,29,57,71,67,57.78689,976,chr1,1561,0.95813253
chr1-100151844-100152202,40,19,24,24,29,15,1,28,19,15,⋯,17,20,24,25,42,38.99721,359,chr1,24,0.34543630


Run Signac prediction

In [8]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 10729411 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100028794-100029147      * |         AGL
         [2]     chr1 100028794-100029147      * |       ALG14
         [3]     chr1 100028794-100029147      * |      CDC14A
         [4]     chr1 100028794-100029147      * |         DBT
         [5]     chr1 100028794-100029147      * |        DPH5
         ...      ...                 ...    ... .         ...
  [10729407]     chrX     9997597-9997902      * |        TLR7
  [10729408]     chrX     9997597-9997902      * |        TLR8
  [10729409]     chrX     9997597-9997902      * |      TMSB4X
  [10729410]     chrX     9997597-9997902      * |     TRAPPC2
  [10729411]     chrX     9997597-9997902      * |        WWC3
                           PeakName               PairName    zscore    pvalue
                        <character>       

In [ ]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna,
                                 matrix.atac,
                                 meta.features,
                                 16)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

In [14]:
execution_time

Time difference of 17.54645 hours

Save results

In [15]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

Warning message in dir.create(dir.output, recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.Naive_CD20_pos_B.240517' already exists”


GRanges object with 10729411 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100028794-100029147      * |         AGL
         [2]     chr1 100029423-100029644      * |         AGL
         [3]     chr1 100037570-100039259      * |         AGL
         [4]     chr1 100046452-100047257      * |         AGL
         [5]     chr1 100063663-100063892      * |         AGL
         ...      ...                 ...    ... .         ...
  [10729407]     chrX   97168682-97168915      * |      NAP1L3
  [10729408]     chrX   97234782-97234932      * |      NAP1L3
  [10729409]     chrX   97373422-97373705      * |      NAP1L3
  [10729410]     chrX   97497924-97498441      * |      NAP1L3
  [10729411]     chrX   97563456-97564399      * |      NAP1L3
                           PeakName               PairName     zscore    pvalue
                        <character>      

In [16]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100028794,100029147,354,*,AGL,chr1-100028794-100029147,chr1-100028794-100029147_AGL,-1.26565616,0.897181894,K562
chr1,100029423,100029644,222,*,AGL,chr1-100029423-100029644,chr1-100029423-100029644_AGL,0.80567855,0.210214110,K562
chr1,100037570,100039259,1690,*,AGL,chr1-100037570-100039259,chr1-100037570-100039259_AGL,0.57796932,0.281642416,K562
chr1,100046452,100047257,806,*,AGL,chr1-100046452-100047257,chr1-100046452-100047257_AGL,-0.12599198,0.550130865,K562
chr1,100063663,100063892,230,*,AGL,chr1-100063663-100063892,chr1-100063663-100063892_AGL,0.17643559,0.429975872,K562
chr1,100064772,100065272,501,*,AGL,chr1-100064772-100065272,chr1-100064772-100065272_AGL,2.52776507,0.005739557,K562
chr1,100065371,100065580,210,*,AGL,chr1-100065371-100065580,chr1-100065371-100065580_AGL,1.49112134,0.067964820,K562
chr1,100132485,100133460,976,*,AGL,chr1-100132485-100133460,chr1-100132485-100133460_AGL,-0.93208269,0.824353105,K562
chr1,100151844,100152202,359,*,AGL,chr1-100151844-100152202,chr1-100151844-100152202_AGL,1.63539885,0.050982731,K562


In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2                     Matrix_1.6-5                     
 [3] BSgenome.Hsapiens.UCSC.hg38_1.4.5 BSgenome_1.